In [1]:
import pandas as pd
import numpy as np
import math as m

paper_ratings = pd.read_csv("../data/ratings.csv")
papers_number, readers_number = paper_ratings.shape

epsilon = 0.000001

paper_steadiness = np.zeros(papers_number)
paper_score = np.zeros(papers_number)
reader_steadiness = np.zeros(readers_number)
reader_score = np.full(readers_number, epsilon)


print(paper_ratings)

# PAPER STEADINESS COMPUTATION

papers_indexes = range(0, papers_number)

for index in papers_indexes:
    
    current_ratings = paper_ratings.iloc[:, index]
    
    for rating in current_ratings:
        
        old_paper_steadiness = paper_steadiness[index]
        old_paper_score = paper_score[index]
        old_reader_steadiness = reader_steadiness[index]
        old_rating_goodness = 1 - (m.sqrt(abs(rating-old_paper_score)))
        old_reader_score = reader_score[index]

   Paper1  Paper2  Paper3
0     0.8     0.0     0.8
1     0.2     0.5     0.0
2     0.2     0.5     0.0
